In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

In [2]:
route = './datasets/'
amount = 3

In [3]:
###########################
# Importar datos benignos #
carpeta_transformados_benign = route +'TransformadosBenign'

# Obtener la lista de archivos pickle en la carpeta
pickle_files = [f for f in os.listdir(carpeta_transformados_benign) if f.endswith('.pickle')]

# Crear una lista vacía para almacenar los dataframes cargados
df_list = []

# Cargar cada archivo pickle en la lista
for file in pickle_files:
    if len(df_list) != amount:
        with open(os.path.join(carpeta_transformados_benign, file), 'rb') as f:
            df = pickle.load(f)
            df_list.append(df)

# Combinar los dataframes cargados en un solo dataframe
df = pd.concat(df_list, ignore_index=True)

In [4]:
df.head()

,udps.n_bytes_per_packet
0,"[[88, 157, 254, 116, 4, 103, 81, 12, 21, 243, ..."
1,"[[120, 131, 191, 232, 32, 48, 89, 92, 52, 223,..."
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"[[60, 139, 25, 116, 70, 94, 194, 11, 248, 250,..."
4,"[[120, 248, 86, 132, 67, 44, 33, 52, 96, 194, ..."


In [5]:
len(df)

114554

In [6]:
###########################
# Importar datos malignos #
carpeta_transformados_malware = route + 'TransformadosMalware'

# Obtener la lista de archivos pickle en la carpeta
pickle_files = [f for f in os.listdir(carpeta_transformados_malware) if f.endswith('.pickle')]

# Crear una lista vacía para almacenar los dataframes cargados
df_doh_list = []

# Cargar cada archivo pickle en la lista
for file in pickle_files:
    if len(df_doh_list) != amount:
        with open(os.path.join(carpeta_transformados_malware, file), 'rb') as f:
            df_doh = pickle.load(f)
            df_doh_list.append(df_doh)

# Combinar los dataframes cargados en un solo dataframe
df_doh = pd.concat(df_doh_list, ignore_index=True)

In [7]:
print("Benign data shape: " + str(df.shape))
print("Malware data shape: " + str(df_doh.shape))

Benign data shape: (114554, 1)
Malware data shape: (63700, 1)


In [8]:
# Cut data
df_small = df.iloc[:10000]
df_doh_small = df_doh.iloc[:10000]

In [10]:
df_small['label'] = np.zeros(len(df_small))
df_doh_small['label'] = np.ones(len(df_doh_small))

C:\Users\Acer\AppData\Local\Temp\ipykernel_4640\231902709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['label'] = np.zeros(len(df_small))
C:\Users\Acer\AppData\Local\Temp\ipykernel_4640\231902709.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_doh_small['label'] = np.ones(len(df_doh_small))


In [11]:
data, data_doh = df_small, df_doh_small
print(data.memory_usage(index=True, deep=True))
print(data_doh.memory_usage(index=True, deep=True))

Index                         128
udps.n_bytes_per_packet    859864
label                       80000
dtype: int64
Index                          128
udps.n_bytes_per_packet    3741656
label                        80000
dtype: int64


In [12]:
full_data = pd.concat([data.sample(frac=1), data_doh.sample(frac=1)]).sample(frac=1)

In [ ]:
#full_data = pd.concat(full_data['udps.n_bytes_per_packet'].apply(lambda x: pd.Series(x)), full_data.iloc[:, -1])

In [13]:
import copy
testData = copy.copy(full_data)
testData = testData['udps.n_bytes_per_packet'].apply(lambda x: pd.Series(x))

In [14]:
# Dividir los datos en conjunto de entrenamiento y prueba (85% entrenamiento, 15% prueba)
train_data, test_data = train_test_split(full_data, test_size=0.15, random_state=42)

# Dividir el conjunto de entrenamiento en conjunto de entrenamiento y validación (70% entrenamiento, 15% validación)
train_data, val_data = train_test_split(train_data, test_size=0.15, random_state=42)

In [15]:
# Obtener las características (X) y etiquetas (y) para cada conjunto de datos
X_train = train_data.iloc[:, :-1]  # Todas las columnas excepto la última
y_train = train_data.iloc[:, -1]  # Última columna (etiquetas)

X_test = test_data.iloc[:, :-1]
y_test = test_data.iloc[:, -1]

X_val = val_data.iloc[:, :-1]
y_val = val_data.iloc[:, -1]

In [16]:
# Verificar las dimensiones de los conjuntos resultantes
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_test:", y_test.shape)
print("Dimensiones de X_val:", X_val.shape)
print("Dimensiones de y_val:", y_val.shape)

Dimensiones de X_train: (14450, 1)
Dimensiones de y_train: (14450,)
Dimensiones de X_test: (3000, 1)
Dimensiones de y_test: (3000,)
Dimensiones de X_val: (2550, 1)
Dimensiones de y_val: (2550,)


In [18]:
clf = TabNetClassifier()  #TabNetRegressor()
clf.fit(
  X_train, y_train,
  eval_set=[(X_val, y_val)]
)
preds = clf.predict(X_test)

TypeError: Pandas DataFrame are not supported: apply X.values when calling fit